In [13]:
import regex
import numpy as np
import pandas as pd
import fasttext
import sys
from gensim.models import KeyedVectors
from gensim.models.wrappers import FastText
from sklearn.metrics import accuracy_score
from sklearn import linear_model
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

`Load Tweets`

In [20]:
""" Copyright 2017, Dimitrios Effrosynidis, All rights reserved. """

import re
from functools import partial
from collections import Counter
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from time import time
import numpy as np
import string




def removeNumbers(text):
    """ Removes integers """
    text = ''.join([i for i in text if not i.isdigit()])         
    return text

def replaceMultiExclamationMark(text):
    """ Replaces repetitions of exlamation marks """
    text = re.sub(r"(\!)\1+", ' multiExclamation ', text)
    return text

def replaceMultiQuestionMark(text):
    """ Replaces repetitions of question marks """
    text = re.sub(r"(\?)\1+", ' multiQuestion ', text)
    return text

def replaceMultiStopMark(text):
    """ Replaces repetitions of stop marks """
    text = re.sub(r"(\.)\1+", ' multiStop ', text)
    return text



def replaceContraction(text):
    patterns = [(re.compile(regex), repl) for (regex, repl) in contraction_patterns]
    for (pattern, repl) in patterns:
        (text, count) = re.subn(pattern, repl, text)
    return text

def replaceElongated(word):
    """ Replaces an elongated word with its basic form, unless the word exists in the lexicon """

    repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
    repl = r'\1\2\3'
    if wordnet.synsets(word):
        return word
    repl_word = repeat_regexp.sub(repl, word)
    if repl_word != word:      
        return replaceElongated(repl_word)
    else:       
        return repl_word

def removeEmoticons(text):
    """ Removes emoticons from text """
    text = re.sub(':\)|;\)|:-\)|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-*|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\*\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:', '', text)
    return text

### Spell Correction begin ###
""" Spell Correction http://norvig.com/spell-correct.html """
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('../data/preprocessing/corporaForSpellCorrection.txt').read()))

def P(word, N=sum(WORDS.values())): 
    """P robability of `word`. """
    return WORDS[word] / N

def spellCorrection(word): 
    """ Most probable spelling correction for word. """
    return max(candidates(word), key=P)

def candidates(word): 
    """ Generate possible spelling corrections for word. """
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    """ The subset of `words` that appear in the dictionary of WORDS. """
    return set(w for w in words if w in WORDS)

def edits1(word):
    """ All edits that are one edit away from `word`. """
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    """ All edits that are two edits away from `word`. """
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

### Spell Correction End ###

### Replace Negations Begin ###

def replace(word, pos=None):
    """ Creates a set of all antonyms for the word and if there is only one antonym, it returns it """
    antonyms = set()
    for syn in wordnet.synsets(word, pos=pos):
        for lemma in syn.lemmas():
            for antonym in lemma.antonyms():
                  antonyms.add(antonym.name())
    if len(antonyms) == 1:
        return antonyms.pop()
    else:
        return None

def replaceNegations(text):
    """ Finds "not" and antonym for the next word and if found, replaces not and the next word with the antonym """
    i, l = 0, len(text)
    words = []
    while i < l:
        word = text[i]
        if word == 'not' and i+1 < l:
            ant = replace(text[i+1])
            if ant:
                words.append(ant)
                i += 2
                continue
        words.append(word)
        i += 1
    return words

### Replace Negations End ###

def addNotTag(text):
    """ Finds "not,never,no" and adds the tag NEG_ to all words that follow until the next punctuation """
    transformed = re.sub(r'\b(?:not|never|no)\b[\w\s]+[^\w\s]', 
       lambda match: re.sub(r'(\s+)(\w+)', r'\1NEG_\2', match.group(0)), 
       text,
       flags=re.IGNORECASE)
    return transformed

def addCapTag(word):
    """ Finds "not,never,no" and adds the tag ALL_CAPS_ to all words that follow until the next punctuation """
    if(len(re.findall("[A-Z]{3,}", word))):
        word = word.replace('\\', '' )
        transformed = re.sub("[A-Z]{3,}", "ALL_CAPS_"+word, word)
        return transformed
    else:
        return word


In [6]:
neg_train = pd.read_fwf('../data/tweets/train_neg.txt', header=None, names=['tweets'])
pos_train = pd.read_fwf('../data/tweets/train_pos.txt', header=None, names=['tweets'])

In [28]:
neg_train.head(50)

,tweets
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...
1,glad i dot have taks tomorrow ! ! #thankful #s...
2,1-3 vs celtics in the regular season = were fu...
3,<user> i could actually kill that girl i'm so ...
4,<user> <user> <user> i find that very hard to ...
5,wish i could be out all night tonight ! <user>
6,<user> i got kicked out the wgm
7,rt <user> <user> <user> yes she is ! u tell it...
8,why is she so perfect <url>
9,<user> hi harry ! did u havea good time in aus...


`Clean Tweets`

In [23]:



stoplist = stopwords.words('english')
# my_stopwords = "multiexclamation multiquestion multistop url atuser st rd nd th am pm" # my extra stopwords
stoplist = stoplist
allowedWordTypes = ["J","R","V","N"] #  J is Adject, R is Adverb, V is Verb, N is Noun. These are used for POS Tagging
lemmatizer = WordNetLemmatizer() # set lemmatizer
stemmer = PorterStemmer() # set stemmer

""" Creates a dictionary with slangs and their equivalents and replaces them """
with open('../data/preprocessing/slang.txt') as file:
    slang_map = dict(map(str.strip, line.partition('\t')[::2]) for line in file if line.strip())

slang_words = sorted(slang_map, key=len, reverse=True) # longest first for regex
regex = re.compile(r"\b({})\b".format("|".join(map(re.escape, slang_words))))
replaceSlang = partial(regex.sub, lambda m: slang_map[m.group(1)])




""" Replaces contractions from a string to their equivalents """
contraction_patterns = [ (r'won\'t', 'will not'), (r'can\'t', 'cannot'), (r'i\'m', 'i am'), (r'ain\'t', 'is not'), (r'(\w+)\'ll', '\g<1> will'), (r'(\w+)n\'t', '\g<1> not'),
                         (r'(\w+)\'ve', '\g<1> have'), (r'(\w+)\'s', '\g<1> is'), (r'(\w+)\'re', '\g<1> are'), (r'(\w+)\'d', '\g<1> would'), (r'&', 'and'), (r'dammit', 'damn it'), (r'dont', 'do not'), (r'wont', 'will not') ]

def tokenize(text):

    sentence = []
    tokens = nltk.word_tokenize(text)
    
    tokens = replaceNegations(tokens) # Technique 6: finds "not" and antonym for the next word and if found, replaces not and the next word with the antonym
    
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator) # Technique 7: remove punctuation

    tokens = nltk.word_tokenize(text) # it takes a text as an input and provides a list of every token in it
                
    tagged = nltk.pos_tag(tokens) # Technique 13: part of speech tagging  
    
    for w in tagged:
        if (w[1][0] in allowedWordTypes and w[0] not in stoplist):
            final_word = addCapTag(w[0])
            #final_word = final_word.lower()
            final_word = replaceElongated(final_word)
            if len(final_word)>1:
                final_word = spellCorrection(final_word)
            final_word = lemmatizer.lemmatize(final_word)
            final_word = stemmer.stem(final_word) 
            sentence.append(final_word)
        
    return sentence

def cleanTweet(text):
    text = replaceSlang(text) # Technique 2: replaces slang words and abbreviations with their equivalents
    text = replaceContraction(text) # Technique 3: replaces contractions to their equivalents
    text = removeNumbers(text) # Technique 4: remove integers from text
    text = replaceMultiExclamationMark(text) # Technique 5: replaces repetitions of exlamation marks with the tag "multiExclamation"
    text = replaceMultiQuestionMark(text) # Technique 5: replaces repetitions of question marks with the tag "multiQuestion"
    text = replaceMultiStopMark(text) # Technique 5: replaces repetitions of stop marks with the tag "multiStop"
    
    tokens = tokenize(text) 
    
    return " ".join(tokens)
    


In [24]:
# clean tweets
neg_train_clean = neg_train['tweets'].apply(cleanTweet)
pos_train_clean = pos_train['tweets'].apply(cleanTweet)

In [35]:
# import pickle

# pickle.dump(neg_train_clean, open("../data/pickle/neg_train_clean.p", 'wb'))
# pickle.dump(pos_train_clean, open("../data/pickle/pos_train_clean.p", 'wb'))

In [80]:

neg_train_clean = pickle.load(open("../data/pickle/neg_train_clean.p", 'rb'))
pos_train_clean = pickle.load(open("../data/pickle/pos_train_clean.p", 'rb'))

def clean_user(text):
    text = text.replace("user", "").replace("url", "").replace("...", "")
    return text.strip()

In [83]:
neg_train_clean = neg_train_clean.apply(clean_user)
pos_train_clean = pos_train_clean.apply(clean_user)

In [84]:
neg_train_clean.head()

0    into tresorpack difficulti object disassembl r...
1                   glad dot take tomorrow thank start
2        versu celtic regular season tuck play playoff
3                               actual kill girl sorri
4                           find hard believ in afraid
Name: tweets, dtype: object

`Load Word Embeddings`

`Note that length of word embeddings for Word2vec and Fasttext is 300 and 200 for Glove.`

`Word2Vec`

In [118]:
word2vec_model = KeyedVectors.load_word2vec_format("data/embeddings/word2vec.en.bin", binary=True)

`Fasttext`

In [37]:
fasttext_model = KeyedVectors.load_word2vec_format("../data/embeddings/crawl-300d-2M.vec",\
                                                   binary=False, limit=100000)
# fasttext_model = FastText.load_fasttext_format('data/embeddings/fasttext.en')
# fasttext_model = fasttext.load_model("data/embeddings/fasttext.1.en.vec")

`Glove`

In [43]:
def loadGloveModel(gloveFile):
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    return model

glove_model = loadGloveModel("../data/embeddings/glove.twitter.27B.200d.txt")

`Pre-processing`

In [85]:
pos_tweets = pos_train_clean.values
neg_tweets = neg_train_clean.values

num_samples = 100000
pos_labels = num_samples * [1] 
neg_labels = num_samples * [0] 

train_sentences = np.array(list(pos_tweets) + list(neg_tweets))
train_labels = np.array(pos_labels + neg_labels)

In [86]:
# average all word-embeddings within a sentence
def average_embeddings(sentences, model, vector_length):
    return_matrix = []
    for sentence in sentences:
        words = sentence.split()
        word_count = float(len(words))
        temp_list = np.zeros(vector_length)
        for word in words:
            try:
                temp_list += model[word][:vector_length]
            except:
                word_count -= 1
        if word_count == 0:
            avrg_list = np.zeros(vector_length)
        else:
            avrg_list = temp_list/word_count
        return_matrix.append(avrg_list)
    return return_matrix

In [87]:
# convert tweets into bag-of-words representations
vector_length = 200
train_features = average_embeddings(train_sentences, glove_model, vector_length)

`Predict Categories with Scikit-learn`

`Logistic Regression Model`

In [88]:
lr = linear_model.LogisticRegression(C=1e5)
lr.fit(train_features, train_labels)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [89]:
accuracy_score(train_labels, lr.predict(train_features))

0.72583500000000001

| `Model` | `Accuracy`   |
|------|------|
| *`Sampled`* `Fasttext` | `0.72694`|
| `Word2vec` | `0.72697`|
| `Glove` | `0.75675`|